In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time

In [2]:
# df=pd.read_csv('~D:/Vasiliev/PERMAK-A/nv/8nom_9dev/out_kk_78.csv', sep=';',names=['tvs', 'tvel', 'time', 'kk'])
# names = glob.glob('D:/Vasiliev/PERMAK-A/nv/*.csv', recursive=True)
# df = pd.concat([pd.read_csv(f,sep=';',names=['tvs', 'tvel', 'time', 'kk']) for i,f in enumerate(names)],
#               ignore_index=True,sort=False)

In [3]:
dfk_origin=pd.read_csv('~D:/Vasiliev/PERMAK-A/nv/8nom_9dev/out_kk_0.csv', sep=';',names=['tvs', 'tvel', 'time', 'kk'])
dfq_origin=pd.read_csv('~D:/Vasiliev/PERMAK-A/nv/8nom_9dev/0.txt', sep=';',names=['kq'])
dfk_origin.loc[:,'tvel']+=1;

In [4]:
#dfk.head(3)
#dfq.head(3)

In [5]:
##############################
### SEARCH FOR HOTTEST TVS ###
##############################

In [7]:
# SET TIMES & INDEXES
dfq_origin['tvs']=1;
dfq_origin['time']=1;
tm=0;
for i in dfq_origin.index:
    dfq_origin.loc[i,'time']=tm;
    dfq_origin.loc[i,'tvs']=i%163+1;
    if ((i+1)%163==0):
        tm+=1;
# SELECT HOTTEST
dfq_origin_first=dfq_origin.loc[0:163]
dfq_origin_first_sorted=dfq_origin_first.sort_values('kq')
dfq_sample=dfq_origin_first_sorted.iloc[143:163]
print(dfq_sample)

        kq  tvs  time
93   1.225   94     0
69   1.226   70     0
54   1.226   55     0
95   1.232   96     0
80   1.232   81     0
68   1.232   69     0
82   1.232   83     0
94   1.232   95     0
67   1.233   68     0
129  1.248  130     0
90   1.249   91     0
141  1.249  142     0
63   1.249   64     0
144  1.249  145     0
135  1.250  136     0
99   1.250  100     0
27   1.251   28     0
18   1.251   19     0
72   1.251   73     0
33   1.252   34     0


In [7]:
###############################
### SEARCH FOR HOTTEST RODS ###
###############################

In [8]:
## SELECT CENTRAL RODS ##
#tvels=np.arange(277,328,step=10)
#s_p=pd.concat([dfk.query('tvs==@indx') for indx in dfqf.index], ignore_index=True, sort=False)
#sample_cen=pd.concat([s_p.query('tvel==@i') for i in tvels], ignore_index=True, sort=False)

In [9]:
## SELECT CORNER RODS ##
#tvels=np.arange(272,331,step=10)
#sample_cor=pd.concat([s_p.query('tvel==@i') for i in tvels], ignore_index=True, sort=False)
#s_p=s_p.drop(s_p.index, inplace=True)

In [10]:
###############################################################################################################################
############################################### END OF SELECTING ##############################################################
###############################################################################################################################

In [10]:
result_cen_dd=pd.DataFrame(None,None,['state','time','tvs','tvel','kk','kq','kr'],None);
result_cor_dd=pd.DataFrame(None,None,['state','time','tvs','tvel','kk','kq','kr'],None);

In [11]:
names_kk_dd = glob.glob('D:/Vasiliev/PERMAK-A/nv/8dev_9dev/9_kk/*.csv', recursive=True)
names_kq_dd = glob.glob('D:/Vasiliev/PERMAK-A/nv/8dev_9dev/9_kq/*.txt', recursive=True)

In [12]:
for i,f in enumerate(names_kk_dd):
    tit1 = time.time()
    ## KQ
    dfq_current=pd.read_csv(names_kq_dd[i], names=['kq'])
    dfq_current['tvs']=1;
    dfq_current['time']=1;
    tm=0;
    for u in dfq_current.index:
        dfq_current.loc[u,'time']=tm;
        dfq_current.loc[u,'tvs']=(u%163)+1;
        if ((u+1)%163==0):
            tm+=1;
    dfq_current_sample=pd.concat([dfq_current.query('tvs==@indx') for indx in (dfq_sample.index+1)], 
                                 ignore_index=True, sort=False)
    ## KK
    dfk_current = pd.read_csv(names_kk_dd[i] ,sep=';',names=['tvs', 'tvel', 'time', 'kk'])
    dfk_current.loc[:,'tvel']+=1;
    ########### OK ###########
    ## SELECT CENTRAL RODS ##
    tvels_cen=np.arange(277,328,step=10)
    s_p=pd.concat([dfk_current.query('tvs==@indx') for indx in (dfq_sample.index+1)], 
                  ignore_index=True, sort=False)
    sample_cen_dd=pd.concat([s_p.query('tvel==@i') for i in tvels_cen], ignore_index=True, sort=False)
    #########################
    ## SELECT CORNER RODS ##
    tvels_cor=np.arange(272,331,step=10)
    sample_cor_dd=pd.concat([s_p.query('tvel==@i') for i in tvels_cor], ignore_index=True, sort=False)
    #########################
    ########## DROPS ########
    s_p=s_p.drop(s_p.index, inplace=True)
    dfq_current.drop(dfq_current.index, inplace=True)
    ##########################
    ####### MAIN CYCLE #######
    for j in sample_cen_dd.index:
        t=len(result_cen_dd)
        v=len(result_cor_dd)
        ## STATE
        result_cen_dd.loc[t,'state']=i;
        result_cor_dd.loc[v,'state']=i;
        ## TIME
        ctime=sample_cen_dd.loc[j,'time'];
        result_cen_dd.loc[t,'time']=ctime;
        result_cor_dd.loc[v,'time']=ctime;
        ## KK & TVEL
        result_cen_dd.loc[t,'kk']=sample_cen_dd.loc[j,'kk'];
        result_cen_dd.loc[t,'tvel']=sample_cen_dd.loc[j,'tvel'];
        result_cor_dd.loc[v,'kk']=sample_cor_dd.loc[j,'kk'];
        result_cor_dd.loc[v,'tvel']=sample_cor_dd.loc[j,'tvel'];
        ## TVS
        ctvs=sample_cen_dd.loc[j,'tvs'];
        result_cen_dd.loc[t,'tvs']=ctvs;
        result_cor_dd.loc[v,'tvs']=ctvs;
        ## KQ
        s=dfq_current_sample.query('tvs==@ctvs & time==@ctime')
        s=pd.to_numeric(s['kq'])
        result_cen_dd.loc[t,'kq']=s.iloc[0];
        result_cor_dd.loc[v,'kq']=s.iloc[0];
        ## KR
        result_cen_dd.loc[t,'kr']=result_cen_dd.loc[t,'kk']*result_cen_dd.loc[t,'kq'];
        result_cor_dd.loc[v,'kr']=result_cor_dd.loc[v,'kk']*result_cor_dd.loc[v,'kq'];
    tit2 = time.time()
    print (i,'->',(tit2-tit1))
    name_to_save_cen = 'D:/Vasiliev/PERMAK-A/nv/8dev_9dev/9_kk/cen1/'
    name_to_save_cor = 'D:/Vasiliev/PERMAK-A/nv/8dev_9dev/9_kk/cor1/'
    name_to_save_cen+=str(i)
    name_to_save_cor+=str(i)
    name_to_save_cen+='.csv'
    name_to_save_cor+='.csv'
    result_cen_dd.to_csv(name_to_save_cen)
    result_cor_dd.to_csv(name_to_save_cor)
    result_cen_dd.drop(result_cen_dd.index, inplace=True)
    result_cor_dd.drop(result_cor_dd.index, inplace=True)
    

0 -> 37.15612506866455
1 -> 37.2441303730011
2 -> 37.432141065597534
3 -> 37.24413013458252
4 -> 37.32913517951965
5 -> 37.370137214660645
6 -> 37.30713391304016
7 -> 37.19512724876404
8 -> 37.38313817977905
9 -> 37.235129594802856
10 -> 37.220128774642944
11 -> 37.337135791778564
12 -> 37.21612858772278
13 -> 37.35613679885864
14 -> 37.5811493396759
15 -> 37.30813384056091
16 -> 37.32513499259949
17 -> 37.26313138008118
18 -> 37.33413577079773
19 -> 37.46414279937744
20 -> 39.241244316101074
21 -> 37.426140785217285
22 -> 39.22424340248108
23 -> 37.452142000198364
24 -> 37.38713836669922
25 -> 37.27913212776184
26 -> 37.27813220024109
27 -> 37.480143785476685
28 -> 37.39613890647888
29 -> 37.41514015197754
30 -> 37.630151987075806
31 -> 37.31213426589966
32 -> 37.22612905502319
33 -> 37.425140380859375
34 -> 37.31813454627991
35 -> 37.28613233566284
36 -> 37.28413271903992
37 -> 37.426140785217285
38 -> 37.52314639091492
39 -> 37.42014026641846
40 -> 37.40213942527771
41 -> 37.3151340

In [7]:
result_cen_nd=pd.DataFrame(None,None,['state','time','tvs','tvel','kk','kq','kr'],None);
result_cor_nd=pd.DataFrame(None,None,['state','time','tvs','tvel','kk','kq','kr'],None);

In [8]:
names_kk_nd = glob.glob('D:/Vasiliev/PERMAK-A/nv/8nom_9dev/9_kk/*.csv', recursive=True)
names_kq_nd = glob.glob('D:/Vasiliev/PERMAK-A/nv/8nom_9dev/9_kq/*.txt', recursive=True)

In [9]:
for i,f in enumerate(names_kk_nd):
    tit1 = time.time()
    ## KQ
    dfq_current=pd.read_csv(names_kq_nd[i], names=['kq'])
    dfq_current['tvs']=1;
    dfq_current['time']=1;
    tm=0;
    for u in dfq_current.index:
        dfq_current.loc[u,'time']=tm;
        dfq_current.loc[u,'tvs']=u%163+1;
        if ((u+1)%163==0):
            tm+=1;
    dfq_current_sample=pd.concat([dfq_current.query('tvs==@indx') for indx in (dfq_sample.index+1)], 
                                 ignore_index=True, sort=False)
    ## KK
    dfk_current = pd.read_csv(names_kk_nd[i] ,sep=';',names=['tvs', 'tvel', 'time', 'kk'])
    dfk_current.loc[:,'tvel']+=1;
    ########### OK ###########
    ## SELECT CENTRAL RODS ##
    tvels_cen=np.arange(277,328,step=10)
    s_p=pd.concat([dfk_current.query('tvs==@indx') for indx in (dfq_sample.index+1)], 
                  ignore_index=True, sort=False)
    sample_cen_nd=pd.concat([s_p.query('tvel==@i') for i in tvels_cen], ignore_index=True, sort=False)
    #########################
    ## SELECT CORNER RODS ##
    tvels_cor=np.arange(272,331,step=10)
    sample_cor_nd=pd.concat([s_p.query('tvel==@i') for i in tvels_cor], ignore_index=True, sort=False)
    #########################
    ########## DROPS ########
    s_p=s_p.drop(s_p.index, inplace=True)
    dfq_current.drop(dfq_current.index, inplace=True)
    ##########################
    ####### MAIN CYCLE #######
    for j in sample_cen_nd.index:
        t=len(result_cen_nd)
        v=len(result_cor_nd)
        ## STATE
        result_cen_nd.loc[t,'state']=i;
        result_cor_nd.loc[v,'state']=i;
        ## TIME
        ctime=sample_cen_nd.loc[j,'time'];
        result_cen_nd.loc[t,'time']=ctime;
        result_cor_nd.loc[v,'time']=ctime;
        ## KK & TVEL
        result_cen_nd.loc[t,'kk']=sample_cen_nd.loc[j,'kk'];
        result_cen_nd.loc[t,'tvel']=sample_cen_nd.loc[j,'tvel'];
        result_cor_nd.loc[v,'kk']=sample_cor_nd.loc[j,'kk'];
        result_cor_nd.loc[v,'tvel']=sample_cor_nd.loc[j,'tvel'];
        ## TVS
        ctvs=sample_cen_nd.loc[j,'tvs'];
        result_cen_nd.loc[t,'tvs']=ctvs;
        result_cor_nd.loc[v,'tvs']=ctvs;
        ## KQ
        s=dfq_current_sample.query('tvs==@ctvs & time==@ctime')
        s=pd.to_numeric(s['kq'])
        result_cen_nd.loc[t,'kq']=s.iloc[0];
        result_cor_nd.loc[v,'kq']=s.iloc[0];
        ## KR
        result_cen_nd.loc[t,'kr']=result_cen_nd.loc[t,'kk']*result_cen_nd.loc[t,'kq'];
        result_cor_nd.loc[v,'kr']=result_cor_nd.loc[v,'kk']*result_cor_nd.loc[v,'kq'];
    tit2 = time.time()
    print (i,'->',(tit2-tit1))
    name_to_save_cen = 'D:/Vasiliev/PERMAK-A/nv/8nom_9dev/9_kk/cen1/'
    name_to_save_cor = 'D:/Vasiliev/PERMAK-A/nv/8nom_9dev/9_kk/cor1/'
    name_to_save_cen+=str(i)
    name_to_save_cor+=str(i)
    name_to_save_cen+='.csv'
    name_to_save_cor+='.csv'
    result_cen_nd.to_csv(name_to_save_cen)
    result_cor_nd.to_csv(name_to_save_cor)
    result_cen_nd.drop(result_cen_nd.index, inplace=True)
    result_cor_nd.drop(result_cor_nd.index, inplace=True)

0 -> 37.60715079307556
1 -> 36.95011353492737
2 -> 36.96611428260803
3 -> 37.35513663291931
4 -> 36.90811085700989
5 -> 37.02811813354492
6 -> 36.99211597442627
7 -> 37.8251633644104
8 -> 37.1671257019043
9 -> 37.637152910232544
10 -> 37.938169717788696
11 -> 38.16818308830261
12 -> 37.95317101478577
13 -> 38.07917809486389
14 -> 38.34819316864014
15 -> 37.84316420555115
16 -> 37.36813712120056
17 -> 37.8321635723114
18 -> 38.15918254852295
19 -> 38.129180908203125
20 -> 37.678155183792114
21 -> 37.343135833740234
22 -> 37.04611897468567
23 -> 37.14112424850464
24 -> 37.06511998176575
25 -> 37.03211808204651
26 -> 37.12912368774414
27 -> 37.08112072944641
28 -> 37.07512068748474
29 -> 37.14912486076355
30 -> 37.1061224937439
31 -> 37.32713484764099
32 -> 37.25313067436218
33 -> 37.11412286758423
34 -> 37.36513710021973
35 -> 37.32613468170166
36 -> 37.22012901306152
37 -> 37.22512936592102
38 -> 37.16012525558472
39 -> 37.2861328125
40 -> 37.33813548088074
41 -> 37.275131940841675
42 -